In [1]:
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from tflearn.layers.recurrent import bidirectional_rnn, BasicLSTMCell
from tflearn.data_utils import to_categorical, pad_sequences
from tflearn.datasets import imdb
import pickle # to save/load objects
import numpy as np
import time, os
import pandas as pd
from networks import * # this has networks

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# IMDB Dataset loading
train, test, _ = imdb.load_data(path='imdb.pkl', n_words=10000, valid_portion=0.15)
trainX, trainY = train
testX, testY = test

# Data preprocessing
# Sequence padding
maxlen = 100
trainX = pad_sequences(trainX, maxlen=maxlen, value=0.)
testX = pad_sequences(testX, maxlen=maxlen, value=0.)
# Converting labels to binary vectors
trainY = to_categorical(trainY, nb_classes=2)
testY = to_categorical(testY, nb_classes=2)

In [4]:
print(sum(trainY)/len(trainY), sum(testY)/len(testY)) # to check if we have balanced sets

[ 0.50047059  0.49952941] [ 0.49733333  0.50266667]


In [5]:
from distutils.version import LooseVersion
import warnings

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. CPU will be used.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.0
Default GPU Device: /gpu:0


In [6]:
# helpers
def test_models(model_list, X, Y):
    accuracies = []
    for m in model_list:
        start = time.time()
        predictions = (np.array(m[1].predict(X))[:,0] >= 0.5).astype(np.int_)
        test_accuracy = np.mean(predictions == Y[:,0], axis=0)
        print("Test accuracy: ", '%.4f' % test_accuracy, 'in', '%.2f' % float(time.time()-start), 'seconds for', m[0])
        accuracies.append(test_accuracy)
    return accuracies

def train_models(model_list, X, Y, epoch, val_set, b_size):
    trianing_times = []
    val_accs = []
    for m in model_list:
        start = time.time()
        temp_callback = CallbackToGetValAcc()
        m[1].fit(X, Y, n_epoch=epoch, validation_set=val_set, show_metric=True, 
                 batch_size=b_size, callbacks=[temp_callback])
        total = float(time.time()-start)
        print("Completed in", '%.2f' % total, 'seconds for', m[0])
        trianing_times.append(total)
        val_accs.append(temp_callback.val)
    return trianing_times, val_accs

In [14]:
start = time.time()
model_conv_do10 = build_conv(maxlen, d_out=None)
model_lstm_do10 = build_lstm(maxlen, d_out=None)
model_conv_do09 = build_conv(maxlen, d_out=0.9)
model_lstm_do09 = build_lstm(maxlen, d_out=0.9)
model_conv_do08 = build_conv(maxlen)
model_lstm_do08 = build_lstm(maxlen)
model_conv_do07 = build_conv(maxlen, d_out=0.7)
model_lstm_do07 = build_lstm(maxlen, d_out=0.7)
model_conv_do06 = build_conv(maxlen, d_out=0.6)
model_lstm_do06 = build_lstm(maxlen, d_out=0.6)
model_conv_do05 = build_conv(maxlen, d_out=0.5)
model_lstm_do05 = build_lstm(maxlen, d_out=0.5)
model_conv_do04 = build_conv(maxlen, d_out=0.4)
model_lstm_do04 = build_lstm(maxlen, d_out=0.4)
model_conv_do03 = build_conv(maxlen, d_out=0.3)
model_lstm_do03 = build_lstm(maxlen, d_out=0.3)
model_conv_do02 = build_conv(maxlen, d_out=0.2)
model_lstm_do02 = build_lstm(maxlen, d_out=0.2)
model_conv_do01 = build_conv(maxlen, d_out=0.1)
model_lstm_do01 = build_lstm(maxlen, d_out=0.1)

models = [
    ['CNN_do10', model_conv_do10], ['RNN_do10', model_lstm_do10],
    ['CNN_do09', model_conv_do09], ['RNN_do09', model_lstm_do09],
    ['CNN_do08', model_conv_do08], ['RNN_do08', model_lstm_do08], 
    ['CNN_do07', model_conv_do07], ['RNN_do07', model_lstm_do07], 
    ['CNN_do06', model_conv_do06], ['RNN_do06', model_lstm_do06], 
    ['CNN_do05', model_conv_do05], ['RNN_do05', model_lstm_do05],
    ['CNN_do04', model_conv_do04], ['RNN_do04', model_lstm_do04],
    ['CNN_do03', model_conv_do03], ['RNN_do03', model_lstm_do03],
    ['CNN_do02', model_conv_do02], ['RNN_do02', model_lstm_do02],
    ['CNN_do01', model_conv_do01], ['RNN_do01', model_lstm_do01]     
         ]

print("Models built in", '%.2f' % float(time.time()-start), 'seconds.')

Models built in 328.01 seconds.


In [15]:
# Hyper-parameters
epoch_size = 8
batch_size = 256
val_size = 0.15

In [16]:
times, val_accs = train_models(models, trainX, trainY, epoch_size, val_size, batch_size)

Training Step: 567  | total loss: 0.29285 | time: 10.297s
| Adam | epoch: 008 | loss: 0.29285 - acc: 0.8870 -- iter: 17920/18062
Training Step: 568  | total loss: 0.29412 | time: 11.456s
| Adam | epoch: 008 | loss: 0.29412 - acc: 0.8854 | val_loss: 0.39314 - val_acc: 0.8269 -- iter: 18062/18062
--
Final model accuracy: 0.826850690013
Completed in 122.58 seconds for RNN_do01


In [17]:
# Testing
accs = test_models(models, testX, testY)

Test accuracy:  0.7957 in 0.53 seconds for CNN_do10
Test accuracy:  0.7912 in 0.40 seconds for RNN_do10
Test accuracy:  0.7981 in 0.29 seconds for CNN_do09
Test accuracy:  0.7968 in 0.57 seconds for RNN_do09
Test accuracy:  0.7883 in 0.29 seconds for CNN_do08
Test accuracy:  0.7971 in 0.58 seconds for RNN_do08
Test accuracy:  0.7979 in 0.28 seconds for CNN_do07
Test accuracy:  0.7976 in 0.59 seconds for RNN_do07
Test accuracy:  0.7864 in 0.29 seconds for CNN_do06
Test accuracy:  0.7984 in 0.58 seconds for RNN_do06
Test accuracy:  0.7992 in 0.28 seconds for CNN_do05
Test accuracy:  0.8040 in 0.58 seconds for RNN_do05
Test accuracy:  0.7944 in 0.29 seconds for CNN_do04
Test accuracy:  0.8000 in 0.60 seconds for RNN_do04
Test accuracy:  0.7893 in 0.29 seconds for CNN_do03
Test accuracy:  0.8043 in 0.59 seconds for RNN_do03
Test accuracy:  0.7915 in 0.28 seconds for CNN_do02
Test accuracy:  0.8144 in 0.59 seconds for RNN_do02
Test accuracy:  0.7888 in 0.29 seconds for CNN_do01
Test accurac

In [18]:
results = pd.DataFrame({'Model':[m[0][:3] for m in models], 
                        'Dropout':[int(m[0][-2:])/10.0 for m in models], 
                        'Train_Time':times, 
                        'Accuracy':accs, 
                        'Val_Acc':val_accs})
results.sort_values(['Accuracy'], ascending=False)

,Accuracy,Dropout,Model,Train_Time,Val_Acc
19,0.824800,0.1,RNN,122.576595,0.826851
17,0.814400,0.2,RNN,134.905302,0.815245
15,0.804267,0.3,RNN,129.761860,0.808344
11,0.804000,0.5,RNN,115.360051,0.803952
13,0.800000,0.4,RNN,114.237031,0.808030
10,0.799200,0.5,CNN,65.898634,0.785132
9,0.798400,0.6,RNN,118.323526,0.793601
2,0.798133,0.9,CNN,64.664051,0.787641
6,0.797867,0.7,CNN,67.589852,0.792660
7,0.797600,0.7,RNN,112.171875,0.795797


In [19]:
# Saving the objects
directory = 'obj'
if not os.path.exists(directory): os.makedirs(directory) # create folder first
f = open('obj/{}.pckl'.format('results_e'+str(epoch_size)), 'wb')
pickle.dump(results, f, protocol=2)
f.close()

In [20]:
DO_Prob = [int(i*10)/10 for i in list(results[results['Model']=='RNN'].Dropout)]
CNN_Val = [int(i*1000)/1000 for i in list(results[results['Model']=='CNN'].Val_Acc)]
RNN_Val = [int(i*1000)/1000 for i in list(results[results['Model']=='RNN'].Val_Acc)]
CNN_Test = [int(i*1000)/1000 for i in list(results[results['Model']=='CNN'].Accuracy)]
RNN_Test = [int(i*1000)/1000 for i in list(results[results['Model']=='RNN'].Accuracy)]
[DO_Prob, CNN_Val, RNN_Val, CNN_Test, RNN_Test]

[[1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1],
 [0.783, 0.787, 0.776, 0.792, 0.778, 0.785, 0.788, 0.785, 0.79, 0.778],
 [0.807, 0.804, 0.795, 0.795, 0.793, 0.803, 0.808, 0.808, 0.815, 0.826],
 [0.795, 0.798, 0.788, 0.797, 0.786, 0.799, 0.794, 0.789, 0.791, 0.788],
 [0.791, 0.796, 0.797, 0.797, 0.798, 0.804, 0.8, 0.804, 0.814, 0.824]]

In [21]:
sum(results.Train_Time)/60

31.066544552644093